# 03. Modeling and Evaluation

## Objective
Implement and compare two models using **NumPy exclusively**:
1.  **Model 1 (OLS)**: Ordinary Least Squares Linear Regression.
2.  **Model 2 (Lasso)**: Lasso Regression (L1 Regularization) using **Coordinate Descent**.

**Evaluation Strategy**:
- **Hold-out**: 20% of data reserved for final testing.
- **Cross-Validation**: 5-Fold CV on the remaining 80% training data.
- **Metrics**: Mean Squared Error (MSE) and $R^2$ Score.

In [155]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
PROCESSED_DATA_PATH = '../data/processed/airbnb_processed.csv'

# Update dtype to reflect scaled float64 columns from preprocessing
processed_dtype = np.dtype([
    ('latitude', np.float64),
    ('longitude', np.float64),
    ('price', np.int32),           # Original Value
    ('minimum_nights', np.float64),
    ('number_of_reviews', np.float64),
    ('reviews_per_month', np.float64),
    ('calculated_host_listings_count', np.float64),
    ('availability_365', np.float64),
    ('price_log', np.float64),     # Target Variable Y
    ('neighbourhood_group_Brooklyn', np.float64),
    ('neighbourhood_group_Manhattan', np.float64),
    ('neighbourhood_group_Queens', np.float64),
    ('neighbourhood_group_Staten Island', np.float64),
    ('room_type_Private room', np.float64),
    ('room_type_Shared room', np.float64),
    ('review_activity', np.float64),
    ('review_quality_score', np.float64),
    ('is_high_value_core', np.float64),
    ('interaction_nights_reviews', np.float64) # New Feature
])

# --- 2. Data Loading Function (Using csv and NumPy) ---

def load_processed_data(file_path, target_dtype):
    with open(file_path, mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader) 
        raw_data = list(reader)
    
    raw_array = np.array(raw_data, dtype=object)
    
    N = len(raw_array)
    structured_data = np.zeros(N, dtype=target_dtype)
    
    for i, name in enumerate(target_dtype.names):
        column_data = raw_array[:, i]
        target_type = target_dtype[name].type
        try:
            structured_data[name] = column_data.astype(target_type)
        except ValueError:
            # Fallback for empty strings if any remain
            column_data[column_data == ''] = '0'
            structured_data[name] = column_data.astype(target_type)
            
    return header, structured_data

header, data = load_processed_data(PROCESSED_DATA_PATH, processed_dtype)

print("--- Data Loading Complete ---")
print(f"Data Shape: {data.shape}")
print(f"Column Names: {data.dtype.names}")

# --- Prepare X (Features) and Y (Target) ---
target_col = 'price_log'
exclude_cols = {'price', 'price_log'}
feature_names = [name for name in data.dtype.names if name not in exclude_cols]

Y_all = data[target_col]
X_list = [data[name] for name in feature_names]
X_all = np.column_stack(X_list)

print(f"\nFeature Matrix X Shape: {X_all.shape}")
print(f"Target Vector Y Shape: {Y_all.shape}")
print(f"Features used: {feature_names}")

--- Data Loading Complete ---
Data Shape: (48258,)
Column Names: ('latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'price_log', 'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan', 'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island', 'room_type_Private room', 'room_type_Shared room', 'review_activity', 'review_quality_score', 'is_high_value_core', 'interaction_nights_reviews')

Feature Matrix X Shape: (48258, 17)
Target Vector Y Shape: (48258,)
Features used: ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan', 'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island', 'room_type_Private room', 'room_type_Shared room', 'review_activity', 'review_quality_score', 'is_high_value_core', 'interaction_nights_rev

## 1. Setup and Splitting
We split the data into **Train (80%)** and **Test (20%)** sets using random shuffling.  
The 20% independent test set will strictly be used ONLY for the final evaluation.

In [156]:
np.random.seed(42)
indices = np.arange(len(X_all))
np.random.shuffle(indices)

test_size = 0.2
split_idx = int(len(X_all) * (1 - test_size))

train_indices = indices[:split_idx]
test_indices = indices[split_idx:]

X_train_full = X_all[train_indices]
Y_train = Y_all[train_indices]

X_test_full = X_all[test_indices]
Y_test = Y_all[test_indices]

print(f"Training Set: {X_train_full.shape[0]} samples")
print(f"Test Set: {X_test_full.shape[0]} samples")

Training Set: 38606 samples
Test Set: 9652 samples


## 2. Models Implementation: Theory & Code

### Ordinary Least Squares (OLS)
OLS seeks to minimize the **Sum of Squared Errors (SSE)**:
$$ J(W) = ||Y - XW||^2_2 $$
This problem has a closed-form solution known as the **Normal Equation**:
$$ W = (X^T X)^{-1} X^T Y $$
In NumPy, we use `np.linalg.lstsq` which uses SVD (Singular Value Decomposition) to solve this, offering better numerical stability than direct matrix inversion.

### Lasso Regression (L1 Regularization)
Lasso adds an L1 penalty term to the objective function to encourage sparsity (feature selection):
$$ J(W) = \frac{1}{2n} ||Y - XW||^2_2 + \lambda ||W||_1 $$
Because the L1 term ($|w|$) is **not differentiable** at $w=0$, subgradient methods can be unstable. The industry standard approach is **Coordinate Descent (Shooting Algorithm)**.

**Coordinate Descent Logic:**
We optimize one regression coefficient $w_j$ at a time while keeping others fixed. The updated value is found using the **Soft Thresholding** operator:
$$ w_j = \frac{S(\rho_j, n\lambda)}{z_j} $$
Where:
- $\rho_j = X_j^T (Y - \hat{Y}_{(-j)}) $ (Correlation between feature $j$ and partial residual)
- $z_j = ||X_j||^2$
- $S(x, \gamma) = \text{sign}(x) \max(|x| - \gamma, 0)$ is the Soft Thresholding function.

This method is robust, converges faster, and correctly sets coefficients to exactly zero.

In [157]:
# --- Feature Preparation ---
def add_intercept(X):
    intercept = np.ones((X.shape[0], 1))
    return np.hstack((intercept, X))

# --- Linear Models ---

def train_linear_regression(X, Y):
    # Uses Normal Equation via SVD (lstsq) for OLS
    W, residuals, rank, singular_values = np.linalg.lstsq(X, Y, rcond=None)
    return W

def predict_linear(X, W):
    return X @ W

# --- Coordinate Descent for Lasso ---

def soft_threshold(rho, lambd):
    """
    Soft Thresholding Operator
    S(rho, lambda) = sign(rho) * max(|rho| - lambda, 0)
    """
    if rho < -lambd:
        return rho + lambd
    elif rho > lambd:
        return rho - lambd
    else:
        return 0.0

def train_lasso_regression(X, Y, lambda_reg=0.01, n_iterations=1000, tol=1e-4):
    """
    Lasso Regression using Coordinate Descent.
    Input lambda_reg corresponds to 'alpha' in sklearn.
    Objective: 1/(2n) * ||y - Xw||^2 + lambda_reg * ||w||_1
    """
    n_samples, n_features = X.shape
    W = np.zeros(n_features)
    
    # Precompute norms of features (denominator z_j)
    # z_j = sum(x_ij^2)
    Z = np.sum(X**2, axis=0)
    
    # Current residuals r = y - Xw (initially y since w=0)
    residuals = Y - X @ W
    
    # Sklearn scales the penalty by n_samples in the threshold check
    # Threshold = n * lambda
    threshold_const = n_samples * lambda_reg
    
    for iteration in range(n_iterations):
        max_w_change = 0.0
        
        for j in range(n_features):
            if Z[j] < 1e-15: continue # Skip zero variance features
            
            # 1. Partial Residual Calculation
            # We want rho_j = X_j^T * (y - y_pred_without_j)
            # y_pred_without_j = y_pred_current - X_j * w_j_current
            # So residuals_without_j = residuals_current + X_j * w_j_current
            
            prev_w = W[j]
            
            # Add feature j back to residual to get partial residual
            # partial_residual = residuals + X[:, j] * prev_w
            # rho_j = X[:, j] @ partial_residual
            # Optimization: rho_j = X_j @ residuals + Z[j] * prev_w
            rho_j = X[:, j] @ residuals + Z[j] * prev_w
            
            # 2. Soft Thresholding Update
            new_w = soft_threshold(rho_j, threshold_const) / Z[j]
            W[j] = new_w
            
            # 3. Update Residuals based on change
            # residuals_new = residuals_old - X_j * (new_w - prev_w)
            w_diff = new_w - prev_w
            if w_diff != 0:
                residuals -= X[:, j] * w_diff
                max_w_change = max(max_w_change, abs(w_diff))
        
        # Check for convergence
        if max_w_change < tol:
            # print(f"Lasso converged at iteration {iteration}")
            break
            
    return W

# --- Metrics ---
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / (ss_tot + 1e-8))

## 3. K-Fold Cross Validation: Logic & Implementation

**Objective**: To robustly estimate model performance and tune hyperparameters ($\lambda$) without touching the Test Set.

**Logic**:
1.  Divide the `X_train_full` data into $k=5$ equal folds.
2.  Iterate $i$ from $0$ to $k-1$:
    -   **Validation Fold**: Fold $i$.
    -   **Training Folds**: All folds except $i$.
3.  Train model on Training Folds $\rightarrow$ Predict on Validation Fold.
4.  Average the MSE/R2 scores across all $k$ iterations.

**NumPy Implementation Details**:
-   We use `np.arange` to generate indices.
-   We slice indices `indices[val_start:val_end]` for validation.
-   We use `np.concatenate` to merge the remaining indices for training.
-   **Note**: We manually add the intercept column inside the loop to ensure clean separation.

In [158]:
def k_fold_cross_validation(X, Y, model_type='linear', k=5, **kwargs):
    """
    Perform K-Fold CV
    X should be passed WITHOUT intercept (Intercept added inside).
    """
    fold_size = len(X) // k
    indices = np.arange(len(X))
    # Data is already shuffled at start, so sequential chunking is fine
    
    mse_scores = []
    r2_scores = []
    
    for i in range(k):
        val_start = i * fold_size
        val_end = (i + 1) * fold_size
        
        val_idx = indices[val_start:val_end]
        train_idx = np.concatenate([indices[:val_start], indices[val_end:]])
        
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]
        
        # Linear models need intercept added explicitly here
        X_train_int = add_intercept(X_train_fold)
        X_val_int = add_intercept(X_val_fold)
        
        if model_type == 'linear':
            W = train_linear_regression(X_train_int, Y_train_fold)
        elif model_type == 'lasso':
            W = train_lasso_regression(X_train_int, Y_train_fold, **kwargs)
        
        Y_pred = predict_linear(X_val_int, W)
        
        mse_val = mse(Y_val_fold, Y_pred)
        r2_val = r2_score(Y_val_fold, Y_pred)
        
        mse_scores.append(mse_val)
        r2_scores.append(r2_val)
        
    return np.mean(mse_scores), np.mean(r2_scores)


def grid_search(X, Y, model_type, param_grid_name, param_values):
    best_score = float('inf')
    best_param = None
    
    print(f"Searching {model_type} with {param_grid_name}: {param_values}")
    
    for val in param_values:
        kwargs = {param_grid_name: val}
        
        if model_type == 'lasso':
            # Coordinate Descent converges fast, can set reasonable max_iter
            kwargs['n_iterations'] = 1000

        avg_mse, avg_r2 = k_fold_cross_validation(X, Y, model_type, k=3, **kwargs)
        
        print(f"  Param {val}: MSE={avg_mse:.4f}, R2={avg_r2:.4f}")
        
        if avg_mse < best_score:
            best_score = avg_mse
            best_param = val
            
    print(f"Best {param_grid_name}: {best_param}")
    return best_param

## 4. Hyperparameter Tuning (Lasso)
We obtain the optimal regularization strength ($\lambda$) for Lasso using the Grid Search defined above. OLS does not have hyperparameters.

In [159]:
# Tune Lasso
lasso_lambdas = [0.0001, 0.001, 0.01, 0.1, 1.0]
best_lambda_lasso = grid_search(X_train_full, Y_train, 'lasso', 'lambda_reg', lasso_lambdas)

Searching lasso with lambda_reg: [0.0001, 0.001, 0.01, 0.1, 1.0]
  Param 0.0001: MSE=0.1920, R2=0.5282
  Param 0.001: MSE=0.1922, R2=0.5276
  Param 0.01: MSE=0.2007, R2=0.5067
  Param 0.1: MSE=0.3013, R2=0.2597
  Param 1.0: MSE=0.4059, R2=0.0026
Best lambda_reg: 0.0001


## 5. Final Evaluation
We now train the final models on the **Entire Training Set (80%)** using the best found parameters (for Lasso). We then evaluate them on the **Independent Test Set (20%)**.

We report:
1.  **MSE (Log Scale)**: The primary metric (loss function).
2.  **R2 Score**: How much variance is explained.
3.  **RMSE (Original Scale)**: For business interpretability ($ USD).

In [ ]:
# Train Final Models on Full Training Set
X_train_int = add_intercept(X_train_full)
X_test_int = add_intercept(X_test_full)

# 1. OLS
W_ols = train_linear_regression(X_train_int, Y_train)
Y_pred_ols = predict_linear(X_test_int, W_ols)

# 2. Lasso (Best)
# Using Coordinate Descent impl
W_lasso = train_lasso_regression(X_train_int, Y_train, lambda_reg=0.0001, n_iterations=1000)
Y_pred_lasso = predict_linear(X_test_int, W_lasso)

# Metrics
mse_ols = mse(Y_test, Y_pred_ols)
mse_lasso = mse(Y_test, Y_pred_lasso)

r2_ols = r2_score(Y_test, Y_pred_ols)
r2_lasso = r2_score(Y_test, Y_pred_lasso)

print("\nFINAL TEST SET RESULTS")
print("=======================================================")
print("{:<15} {:<15} {:<10}".format("Model", "MSE (Log)", "R2 Score"))
print("-" * 40)
print("{:<15} {:<15.4f} {:<10.4f}".format("OLS (Linear)", mse_ols, r2_ols))
print("{:<15} {:<15.4f} {:<10.4f}".format(f"Lasso (l={best_lambda_lasso})", mse_lasso, r2_lasso))

# RMSE Original Scale Calculation (USD)
# Best model select
models = {'OLS': mse_ols, 'Lasso': mse_lasso}
best_model_name = min(models, key=models.get)

if best_model_name == 'OLS': best_pred = Y_pred_ols
elif best_model_name == 'Lasso': best_pred = Y_pred_lasso

# Transform back: exp(log_price) - 1
Y_test_orig = np.exp(Y_test) - 1
Y_pred_orig = np.exp(best_pred) - 1

rmse_price = np.sqrt(mse(Y_test_orig, Y_pred_orig))



FINAL TEST SET RESULTS
Model           MSE (Log)       R2 Score  
----------------------------------------
OLS (Linear)    0.1927          0.5319    
Lasso (l=0.0001) 0.1988          0.5172    


## 6. Verification: Comparison with Scikit-learn

In this final section, we verify the correctness of our custom **NumPy** implementations by comparing them against the industry-standard **Scikit-learn** versions. 

- We rely on `sklearn.linear_model` for the models.
- We continue to use our **custom NumPy metrics** (`mse`, `r2_score`) for evaluation to ensure an apples-to-apples comparison.

In [161]:
from sklearn.linear_model import LinearRegression, Lasso

print("--- Scikit-learn Verification ---\n")

# 1. Scikit-learn OLS
sklearn_ols = LinearRegression()
sklearn_ols.fit(X_train_full, Y_train)
Y_pred_sklearn_ols = sklearn_ols.predict(X_test_full)

mse_sklearn_ols = mse(Y_test, Y_pred_sklearn_ols)
r2_sklearn_ols = r2_score(Y_test, Y_pred_sklearn_ols)
rmse_sklearn_ols = np.sqrt(mse(np.exp(Y_test)-1, np.exp(Y_pred_sklearn_ols)-1))

# 2. Scikit-learn Lasso (Using same best_lambda_lasso)
# Note: Sklearn alpha is our lambda. Sklearn optimizes 1/(2n) * ||y - Xw||^2 + alpha * ||w||_1
# Our objective function was the same, so parameter should match directly.
sklearn_lasso = Lasso(alpha=best_lambda_lasso)
sklearn_lasso.fit(X_train_full, Y_train)
Y_pred_sklearn_lasso = sklearn_lasso.predict(X_test_full)

mse_sklearn_lasso = mse(Y_test, Y_pred_sklearn_lasso)
r2_sklearn_lasso = r2_score(Y_test, Y_pred_sklearn_lasso)
rmse_sklearn_lasso = np.sqrt(mse(np.exp(Y_test)-1, np.exp(Y_pred_sklearn_lasso)-1))

# Calculate RMSE for Custom models for the table
rmse_ols = np.sqrt(mse(np.exp(Y_test)-1, np.exp(Y_pred_ols)-1))
rmse_lasso = np.sqrt(mse(np.exp(Y_test)-1, np.exp(Y_pred_lasso)-1))

print("=================================================================================")
print("FINAL COMPARISON: CUSTOM NUMPY vs SCIKIT-LEARN")
print("=================================================================================")
print("{:<10} {:<15} {:<15} {:<10} {:<15}".format("Model", "Implementation", "MSE (Log)", "R2", "RMSE (Orig $)"))
print("-" * 75)

print("{:<10} {:<15} {:<15.4f} {:<10.4f} ${:<15.2f}".format("OLS", "Custom NumPy", mse_ols, r2_ols, rmse_ols))
print("{:<10} {:<15} {:<15.4f} {:<10.4f} ${:<15.2f}".format("OLS", "Scikit-learn", mse_sklearn_ols, r2_sklearn_ols, rmse_sklearn_ols))
print("-" * 75)
print("{:<10} {:<15} {:<15.4f} {:<10.4f} ${:<15.2f}".format("Lasso", "Custom NumPy", mse_lasso, r2_lasso, rmse_lasso))
print("{:<10} {:<15} {:<15.4f} {:<10.4f} ${:<15.2f}".format("Lasso", "Scikit-learn", mse_sklearn_lasso, r2_sklearn_lasso, rmse_sklearn_lasso))
print("=================================================================================")

--- Scikit-learn Verification ---

FINAL COMPARISON: CUSTOM NUMPY vs SCIKIT-LEARN
Model      Implementation  MSE (Log)       R2         RMSE (Orig $)  
---------------------------------------------------------------------------
OLS        Custom NumPy    0.1927          0.5319     $80.38          
OLS        Scikit-learn    0.1927          0.5319     $80.38          
---------------------------------------------------------------------------
Lasso      Custom NumPy    0.1988          0.5172     $81.47          
Lasso      Scikit-learn    0.1928          0.5319     $80.43          
